In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import ezhc as hc

from pandas.tseries.offsets import BDay
from IPython.display import display
from tools import make_track, ols_regression

In [4]:
prices = pd.read_csv(r"financial_data/prices.csv", index_col=0)
prices.index = pd.DatetimeIndex(prices.index)

mondays = pd.date_range(start=dt.date(2010, 1, 4), end=dt.date.today(), freq='7D')
returns = prices.reindex(mondays).ffill().pct_change().dropna()

sx5e = returns[["SX5E"]]
bch = returns.drop("SX5E", axis=1)

size = 52
freq = 13

df_weight = ols_regression(sx5e, bch, size, freq)
prices_for_track = prices.loc[df_weight.index[0]:].drop("SX5E", axis=1)
replication = make_track(prices_for_track, df_weight)

df_res = prices.loc[replication.index][["SX5E"]]
df_res["OLS"] = replication

df_res = df_res / df_res.iloc[0]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [5]:
df_weight

,IBEX,DAX,FSTEMIB,CAC 40
2011-01-10,0.065454,0.546487,0.188874,0.190240
2011-04-11,0.115589,0.524407,0.167010,0.181889
2011-07-11,0.105270,0.522633,0.198118,0.198708
2011-10-10,0.150829,0.430029,0.261294,0.183924
2012-01-09,0.074195,0.357945,0.338733,0.237828
2012-04-09,0.090943,0.361514,0.359649,0.191175
2012-07-09,0.106874,0.366799,0.371657,0.150153
2012-10-08,0.072935,0.426972,0.352011,0.151273
2013-01-07,0.041771,0.461200,0.359344,0.153989
2013-04-08,0.113653,0.424125,0.350509,0.121131


In [17]:
df_weight.sum(axis=1)

2011-01-10    0.991055
2011-04-11    0.988895
2011-07-11    1.024728
2011-10-10    1.026076
2012-01-09    1.008700
2012-04-09    1.003280
2012-07-09    0.995482
2012-10-08    1.003191
2013-01-07    1.016304
2013-04-08    1.009418
2013-07-08    1.002486
2013-10-07    1.002821
2014-01-06    1.010223
2014-04-07    1.003865
2014-07-07    1.001201
2014-10-06    0.987671
2015-01-05    1.006990
2015-04-06    1.008657
2015-07-06    1.037528
2015-10-05    1.031914
2016-01-04    1.024537
2016-04-04    1.004851
2016-07-04    1.023431
2016-10-03    1.026952
2017-01-02    1.036558
2017-04-03    1.052425
2017-07-03    0.828264
dtype: float64

In [10]:
df_res.loc["28-6-2017":].head(10)

,SX5E,OLS
2017-06-28,1.280642,2.015385
2017-06-29,1.257327,1.979377
2017-06-30,1.257327,1.964677
2017-07-03,1.264745,1.649321
2017-07-04,1.260276,1.644938
2017-07-05,1.259892,1.643483
2017-07-06,1.259892,1.636493
2017-07-07,1.254615,1.637773
2017-07-10,1.259765,1.644755
2017-07-11,1.254846,1.641724


In [7]:
g = hc.Highstock()

g.chart.height = 550
g.legend.enabled = True
g.legend.layout = 'horizontal'
g.legend.align = 'center'
g.legend.maxHeight = 100
g.tooltip.enabled = True
g.tooltip.valueDecimals = 2
g.exporting.enabled = True

g.chart.zoomType = 'xy'
g.title.text = 'OLS regression on SX5E'

g.plotOptions.series.compare = 'percent'
g.yAxis.labels.formatter = hc.scripts.FORMATTER_PERCENT
g.tooltip.pointFormat = hc.scripts.TOOLTIP_POINT_FORMAT_PERCENT
g.tooltip.positioner = hc.scripts.TOOLTIP_POSITIONER_CENTER_TOP

g.xAxis.gridLineWidth = 1.0
g.xAxis.gridLineDashStyle = 'Dot'
g.yAxis.gridLineWidth = 1.0
g.yAxis.gridLineDashStyle = 'Dot'

g.series = hc.build.series(df_res)


g.plot_with_table_1(save=False, save_name='NoTable')

In [8]:
[2]*5

[2, 2, 2, 2, 2]

In [10]:
x = dt.datetime(2017,11,18)
x + BDay(0)

Timestamp('2017-11-20 00:00:00')